[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/marmalux/Clientes_TelecomX/blob/main/Clientes_TelecomX.ipynb)


# Análisis de evasión de clientes en Telecom X

## Importación de datos

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json

#importar de una API y pasar a json
url = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json'
datos_api = requests.get(url)
datos_json=json.loads(datos_api.text)

In [2]:
# normalizar datos
datos_cliente = pd.json_normalize(datos_json)
datos_cliente.sample(5)

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
1934,2737-WFVYW,Yes,Female,0,No,No,2,No,No phone service,DSL,...,No,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,45.25,85.5
5969,8160-HOWOX,No,Female,0,No,No,7,Yes,No,DSL,...,No,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,66.85,458.1
4338,5955-EPOAZ,No,Female,0,No,No,6,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,Yes,Mailed check,20.95,109.5
4554,6242-MBHPK,No,Female,1,Yes,No,23,Yes,No,Fiber optic,...,No,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,95.10,2326.05
4952,6765-MBQNU,No,Female,0,Yes,No,26,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,26.00,684.05


### Diccionario de datos

- `customerID`: número de identificación único de cada cliente
- `Churn`: si el cliente dejó o no la empresa
- `gender`: género (masculino y femenino)
- `SeniorCitizen`: información sobre si un cliente tiene o no una edad igual o mayor a 65 años
- `Partner`: si el cliente tiene o no una pareja
- `Dependents`: si el cliente tiene o no dependientes
- `tenure`: meses de contrato del cliente
- `PhoneService`: suscripción al servicio telefónico
- `MultipleLines`: suscripción a más de una línea telefónica
- `InternetService`: suscripción a un proveedor de internet
- `OnlineSecurity`: suscripción adicional de seguridad en línea
- `OnlineBackup`: suscripción adicional de respaldo en línea
- `DeviceProtection`: suscripción adicional de protección del dispositivo
- `TechSupport`: suscripción adicional de soporte técnico, menor tiempo de espera
- `StreamingTV`: suscripción de televisión por cable
- `StreamingMovies`: suscripción de streaming de películas
- `Contract`: tipo de contrato
- `PaperlessBilling`: si el cliente prefiere recibir la factura en línea
- `PaymentMethod`: forma de pago
- `Charges.Monthly`: total de todos los servicios del cliente por mes
- `Charges.Total`: total gastado por el cliente

## Transformación de datos

Valores unicos

In [17]:
for columna in datos_cliente:
  if datos_cliente[columna].nunique() <50:
    print(f'Valores unicos de la columna {columna} \n{datos_cliente[columna].unique()}')
    print(f'-'*50)


Valores unicos de la columna Churn 
['No' 'Yes' '']
--------------------------------------------------
Valores unicos de la columna customer.gender 
['Female' 'Male']
--------------------------------------------------
Valores unicos de la columna customer.SeniorCitizen 
[0 1]
--------------------------------------------------
Valores unicos de la columna customer.Partner 
['Yes' 'No']
--------------------------------------------------
Valores unicos de la columna customer.Dependents 
['Yes' 'No']
--------------------------------------------------
Valores unicos de la columna phone.PhoneService 
['Yes' 'No']
--------------------------------------------------
Valores unicos de la columna phone.MultipleLines 
['No' 'Yes' 'No phone service']
--------------------------------------------------
Valores unicos de la columna internet.InternetService 
['DSL' 'Fiber optic' 'No']
--------------------------------------------------
Valores unicos de la columna internet.OnlineSecurity 
['No' 'Yes' 'N

Datos duplicados

In [3]:
datos_cliente.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7267 non-null   object 
 1   Churn                      7267 non-null   object 
 2   customer.gender            7267 non-null   object 
 3   customer.SeniorCitizen     7267 non-null   int64  
 4   customer.Partner           7267 non-null   object 
 5   customer.Dependents        7267 non-null   object 
 6   customer.tenure            7267 non-null   int64  
 7   phone.PhoneService         7267 non-null   object 
 8   phone.MultipleLines        7267 non-null   object 
 9   internet.InternetService   7267 non-null   object 
 10  internet.OnlineSecurity    7267 non-null   object 
 11  internet.OnlineBackup      7267 non-null   object 
 12  internet.DeviceProtection  7267 non-null   object 
 13  internet.TechSupport       7267 non-null   objec

In [5]:
print("Numero de duplicados", datos_cliente.duplicated().sum())

Numero de duplicados 0


Valores nulos

In [7]:
print("Numero de nulos", datos_cliente.isnull().sum())

Numero de duplicados customerID                   0
Churn                        0
customer.gender              0
customer.SeniorCitizen       0
customer.Partner             0
customer.Dependents          0
customer.tenure              0
phone.PhoneService           0
phone.MultipleLines          0
internet.InternetService     0
internet.OnlineSecurity      0
internet.OnlineBackup        0
internet.DeviceProtection    0
internet.TechSupport         0
internet.StreamingTV         0
internet.StreamingMovies     0
account.Contract             0
account.PaperlessBilling     0
account.PaymentMethod        0
account.Charges.Monthly      0
account.Charges.Total        0
dtype: int64


Buscar datos vacios no nulos

In [ ]:


# print(datos_cliente['Churn'].unique()) # en la columna Churn hay valores incoherentes

# datos_cliente['Churn'].str.strip()
(datos_cliente[(datos_cliente['Churn'] == '')].head())
# (datos_cliente[(datos_cliente['account.Charges.Total'] == ' ')].head())


Después de revisar, la columna *Churn* y la columna *account.Charges.Total* tienen valores inconsistentes, *Churn* faltan  valores de 'Yes' pues depués de analizar, hay gastos totales por lo que debe ser aun cliente de la empresa, y la columna de gastos totales *account.Charges.Total* tiene valores vacios no nulos, pero como son clientes que ya no estan en la empresa se puden dejar como 0 para poder cambiar el tipo de dato

In [ ]:
# transformar columna Churn
datos_cliente['Churn'] = datos_cliente['Churn'].replace('',pd.NA).fillna('No')# se remplazan las celdas vacias a valores nulos y despues se rellenan esos valores con 'No'

#transformar columna 	account.Charges.Total
datos_cliente['account.Charges.Total'] = datos_cliente['account.Charges.Total'].replace(' ',pd.NA).fillna(0)

In [ ]:
#Pasar a tipo float los datos de los cargos totales
datos_cliente['account.Charges.Total']=datos_cliente['account.Charges.Total'].astype(np.float64)

In [ ]:
datos_cliente.info()

### Formatear nombres de columnas para mejor entendimiento en presentación

In [ ]:
columnas=['customerID',
          'Still_Customer',
          'Gender',
          'Senior_Citizen',
          'Partner',
          'Dependents',
          'Contract_Months',
          'Phone_Service',
          'Multiple_Lines',
          'Internet_Service',
          'Online_Security',
          'Online_Backup',
          'Device_Protection',
          'Tech_Support',
          'Streaming_TV',
          'Streaming_Movies',
          'Contract_type',
          'Paperless_Billing',
          'Payment_Method',
          'Charges_Monthly',
          'Charges_Total']
datos_cliente.columns = columnas

Crear columa de cargos al día "Charges_Daily" y una columna de cantidad de servicios contratados

In [ ]:
#
gastos_diarios=datos_cliente['Charges_Monthly'].to_numpy()
gastos_diarios = gastos_diarios/30
datos_cliente['Charges_Day'] = np.round(gastos_diarios,2)

In [ ]:
datos_cliente.sample(5)

Hay columnas en las cuales solo existen los valores unicos de "Yes" y "No" , por lo que se pueden cambiar valores a 0 y 1 para poder hacer operaciones mas sencillas, hay otros que tienen mas valores unicos que no se pueden cambiar tan facilmente, así que el análisis será distinto

Las siguientes columans son las que pueden cambiar a eso
 * 'Still_Customer'
 * 'Partner'
 * 'Dependents'
 * 'Phone_Service'
 * 'Paperless_Billing'
 * 'Mutiple_Lines'

In [ ]:
datos_cliente[['Still_Customer',
                'Partner',
                'Dependents',
                'Phone_Service',
                'Paperless_Billing']]=datos_cliente[['Still_Customer',
                                      'Partner',
                                      'Dependents',
                                      'Phone_Service',
                                      'Paperless_Billing']].replace('No',0).replace('Yes',1);
# cambiar el tipo de dato de estas columnas para poder trabajar con ellas como enteros
datos_cliente[['Still_Customer',
                'Partner',
                'Dependents',
                'Phone_Service',
                'Paperless_Billing']]\
                =datos_cliente[['Still_Customer',
                                'Partner',
                                'Dependents',
                                'Phone_Service',
                                'Paperless_Billing']].astype(int);

In [ ]:
datos_cliente.info()

## Análisis descriptivo de valores

In [ ]:
estadisticas_columnas=datos_cliente.describe() # se almacena el dataframe en una variable para poder usar estos datos despues
datos_cliente.describe()

Para los columnas donde hay mas que "Yes" y "No" como 'Gender' o 'Internet_Service

In [ ]:
porcentaje_genero = (datos_cliente['Gender']=='Female').mean()
servicios_internet = datos_cliente['Internet_Service'].unique()

cantidad_servicios_internet= [(datos_cliente['Internet_Service']==i).sum() for i in servicios_internet]
# porcentaje_servicio_internet = (datos_cliente['Internet_Service']=='DSL').mean()
plt.pie(cantidad_servicios_internet,labels = servicios_internet, autopct="%0.1f %%")
plt.title('Servicios de internet')
plt.show()

Como son la cantidad de clientes que ya no estan los que necesitamos para responder, se observa como es la distrubucion de esta

In [ ]:
fig, ax=plt.subplots()
distribucion_cliente =estadisticas_columnas.loc['mean','Still_Customer']
ax.pie(x=[distribucion_cliente,1-distribucion_cliente],labels=['Si','No'],autopct='%0.2f%%')
ax.set_title('Distribucion de clientes, ¿aun es cliente?')
plt.show()

### Recuento por variables categoricas

Recuento por variables categoricas como género, acompañante, si es mayor a 65, etc para saber si hay algo que determine o oriente al resultado de la baja de clientes

In [ ]:
# Categorias que pueden ser cercanas a lo relacionado con asuntos de las personas como el genero, acompañantes o si es una persona mayor


categorias_genero=['Senior_Citizen', 'Partner','Dependents']
fig,axs=plt.subplots(nrows=len(categorias_genero),ncols=2,figsize=(10,3.5*len(categorias_genero)))

#se crean las graficas observando el comportamiento de los que son y no son clientes para estas categorias divididas por genero
for cliente in datos_cliente['Still_Customer'].unique():
  for categorico in categorias_genero:
    # Se crea una tabla dinamica para cada categoria y despues su grafica de barras
    tabla= pd.pivot_table(data = datos_cliente[datos_cliente['Still_Customer']==cliente],index='Gender',columns=categorico,aggfunc='size')

    #la grafica se coloca en el espacio asignado para la comparacion segun el subplot
    tabla.plot(kind='bar',stacked=False,title=f'Comparacion de {'No Clientes' if cliente==0 else 'Clientes'} entre genero y {categorico}'\
               ,ax=axs[categorias_genero.index(categorico),cliente]);


for ax in axs.flat:
  ax.set_ylabel('Cantidad de usuarios') # se coloca el titulo del eje Y
plt.tight_layout()# la grafica se estiliza a la ventana
plt.show()

In [ ]:
# categorias aparte que dependen mas del servicio
categorias_varias=['Internet_Service','Phone_Service','Contract_type', 'Paperless_Billing']

fig,axs=plt.subplots(nrows=len(categorias_varias),figsize=(5,3.5*len(categorias_varias)))

for categorico in categorias_varias:
  tabla= pd.pivot_table(data =  datos_cliente,index='Still_Customer',columns=categorico,aggfunc='size')

  tabla.plot(kind='bar',stacked=False,title=f'Comparacion de los que son y ya no son Clientes y {categorico}'\
              ,ax=axs[categorias_varias.index(categorico)]); # se colocan las posiciones de la grafica en un lugar del subplot

  axs[categorias_varias.index(categorico)].set_xticklabels(['No','Si']) # se formatea las etiquetas del eje x para que sea mejor visualmente

for ax in axs.flat:
  ax.tick_params(axis='x', rotation=0) # se cambia la direccion de las equiquetas del eje x a 0 para mejor visualizacion
  ax.set_ylabel('Cantidad de usuarios')

plt.tight_layout()# la grafica se estiliza a la ventana
plt.show()

### Conteo de evasión de clientes por variables numéricas

Se hace una comparativa de los gastos de los usuarios que son y no son clientes para examinar si existe una tendencia, para ello se usa el metodo de pago como referencia

In [ ]:

cargos = ['Charges_Monthly','Charges_Day']
fig,axs=plt.subplots(2,2,figsize=(10,6))
for cliente in datos_cliente['Still_Customer'].unique():
  for i,cargo in enumerate(cargos):
    axs[cliente,i].hist(datos_cliente[datos_cliente['Still_Customer']==cliente][cargo],bins=15)
    axs[cliente,i].set_title(f'Incidencias de precios por {'Mes' if i==0 else 'Dia'} para {'No Clientes' if cliente==0 else 'Clientes'}')
    axs[cliente,i].set_xlabel(f'Pago por {'Mes' if i==0 else 'Dia'}')

plt.tight_layout()# la grafica se estiliza a la ventana
plt.show()

In [ ]:
categorias_numericas=['Internet_Service','Phone_Service','Multiple_Lines']
fig, axs=plt.subplots(nrows=len(categorias_numericas),ncols=2,figsize=(7,len(categorias_numericas)*3.5))
for cliente in datos_cliente['Still_Customer'].unique():
  for i,categoria in enumerate(categorias_numericas):
    tabla = pd.pivot_table(data=datos_cliente[datos_cliente['Still_Customer']==cliente],index=categoria,values='Charges_Day',aggfunc='mean')
    tabla.plot(kind='bar',title=f'Pago promedio de {'No cliente' if cliente==0 else 'Cliente'} por dia ',\
               ax=axs[i,cliente],rot=0)

for ax in axs.flat:
  ax.legend('')
plt.tight_layout()
plt.show()

In [ ]:
categorias_numericas=['Internet_Service','Phone_Service','Multiple_Lines']
fig, axs=plt.subplots(nrows=len(categorias_numericas),ncols=2,figsize=(7,len(categorias_numericas)*3.5))
for cliente in datos_cliente['Still_Customer'].unique():
  for i,categoria in enumerate(categorias_numericas):
    tabla = pd.pivot_table(data=datos_cliente[datos_cliente['Still_Customer']==cliente],index=categoria,values='Charges_Total',aggfunc='mean')
    tabla.plot(kind='bar',title=f'Pago promedio total de {'No cliente' if cliente==0 else 'Cliente'}',\
               ax=axs[i,cliente],rot=0)

for ax in axs.flat:
  ax.legend('')

plt.tight_layout()
plt.show()

Meses de contrato para los usuarios

In [ ]:
fig, axs=plt.subplots(nrows=len(categorias_numericas),ncols=2,figsize=(7,len(categorias_numericas)*3.5))
for cliente in datos_cliente['Still_Customer'].unique():
  for i,categoria in enumerate(categorias_numericas):
    tabla = pd.pivot_table(data=datos_cliente[datos_cliente['Still_Customer']==cliente],index=categoria,values='Contract_Months',aggfunc='mean')
    tabla.plot(kind='bar',title=f'Meses de contrato promedio de {'No cliente' if cliente==0 else 'Cliente'}\npor servicios ',\
               ax=axs[i,cliente],rot=0)

for ax in axs.flat:
  ax.legend('')

plt.tight_layout()
plt.show()

In [ ]:
# Correlacion
datos_cliente[['Still_Customer','Contract_Months','Charges_Monthly','Charges_Total','Charges_Day']].corr()

In [ ]:
datos_cliente.head()

In [ ]:
(datos_cliente.loc[:,'Online_Security':'Streaming_Movies']== 'Yes').sum(axis=1)

In [ ]:
dat

## Informe Final